# IMDB verisi üzerinde Bidirectional LSTM'in Basit Bir Uygulaması

Bu notebook'da IMDB film inceleme duyarlılığı sınıflandırma veri kümesinde 2 katmanlı çift yönlü bir LSTM eğiteceğiz.

## Kurulum

In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

max_features = 20000  # Yalnızca ilk 20 bin kelimeyi dikkate alın
maxlen = 200  # Her film incelemesinin yalnızca ilk 200 kelimesini dikkate alın


## Modeli oluşturalım

In [2]:
# Değişken uzunluklu tamsayı dizileri için girdi
inputs = keras.Input(shape=(None,), dtype="int32")
# Her tamsayıyı 128 boyutlu bir vektöre gömün
x = layers.Embedding(max_features, 128)(inputs)
# 2 çift yönlü LSTM ekleyin
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
# Sınıflandırıcı ekle
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 128)         2560000   
                                                                 
 bidirectional (Bidirectiona  (None, None, 128)        98816     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 2,757,761
Trainable params: 2,757,761
Non-train

## IMDB film incelemesi için duyarlılık verilerini yükleyin

In [3]:
(x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(
    num_words=max_features
)
print(len(x_train), "Eğitim dizileri")
print(len(x_val), "Doğrulama dizileri")
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_val = keras.preprocessing.sequence.pad_sequences(x_val, maxlen=maxlen)


17464789/17464789 [==============================] - 2s 0us/step
25000 Eğitim dizileri
25000 Doğrulama dizileri


## Modeli eğitin ve değerlendirin

Ayrıca [Hugging Face Hub](https://huggingface.co/keras-io/bidirectional-lstm-imdb) üzerinde barındırılan eğitimli modeli kullanabilirdi..

In [4]:
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=32, epochs=2, validation_data=(x_val, y_val))


Epoch 1/2
782/782 [==============================] - 111s 126ms/step - loss: 0.3809 - accuracy: 0.8272 - val_loss: 0.3284 - val_accuracy: 0.8629
Epoch 2/2
782/782 [==============================] - 44s 57ms/step - loss: 0.1963 - accuracy: 0.9267 - val_loss: 0.3373 - val_accuracy: 0.8545


## Model parametrelerini optimize edelim

In [ ]:
# model parametrelerin optimize edilmesi.
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

def create_model(optimizer='adam'):
    model = keras.Sequential()
    model.add(layers.Embedding(max_features, 128))
    model.add(layers.Bidirectional(layers.LSTM(64, return_sequences=True)))
    model.add(layers.Bidirectional(layers.LSTM(64)))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, verbose=0)
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [2, 3, 5, 10]
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(batch_size=batch_size, epochs=epochs, optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(x_train, y_train)

# en iyi parametreler
print("En iyi parametreler: %s with a score of %f" % (grid_result.best_params_, grid_result.best_score_))

# en iyi parametreler ile modelin yeniden eğitilmesi
model = create_model(optimizer = 'Adam')
model.fit(x_train, y_train, batch_size=100, epochs=10, validation_data=(x_val, y_val))

# modelin test verileri ile test edilmesi
model.evaluate(x_val, y_val)